In [96]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [97]:
import cupy as np
import cudf as pd

In [98]:
train_wine = pd.read_csv('../input/ammi-bootcamp-kaggle-competition/train.csv')
print('Train data successfully loaded...!!!')
test_wine = pd.read_csv('../input/ammi-bootcamp-kaggle-competition/test.csv')
print('Test data successfully loaded...!!!')

In [99]:
train_wine.set_index('id',inplace=True)
test_wine.set_index('id',inplace=True)

In [100]:
test_wine.info()

In [101]:
train_wine['points'] = train_wine['points'].astype(int)
test_wine['points'] = test_wine['points'].astype(int)

In [102]:
p = ['title','taster_twitter_handle','taster_name']
q = ['index','title','taster_twitter_handle','taster_name','price']
train_wine.drop(columns=p, axis =1,inplace=True)
test_wine.drop(columns=q, axis =1,inplace=True)

In [103]:
train_wine.head()

In [104]:
test_wine.head()

In [105]:
train_wine['points'].describe()

In [106]:
from collections import Counter
Counter(train_wine['points'].to_pandas())

In [107]:
print('The missing values in the training data are as shown below:')
train_wine.isnull().sum().sort_values(ascending=False)

In [108]:
print('The missing values in the test data are as shown below:')
test_wine.isnull().sum().sort_values(ascending=False)

In [109]:
train_wine.drop('region_2',axis=1,inplace=True)
test_wine.drop('region_2',axis=1,inplace=True)

In [110]:
for i in train_wine.columns:
  if train_wine[i].isnull().sum() >= 1:
    train_wine[i].fillna('missing',inplace = True)
train_wine.isnull().sum().sort_values(ascending=False)    

In [111]:
for i in test_wine.columns:
  if test_wine[i].isnull().sum() >= 1:
    test_wine[i].fillna('missing',inplace = True)
test_wine.isnull().sum().sort_values(ascending=False)    

In [112]:
print(train_wine.shape)
print(test_wine.shape)

In [113]:
train_wine.dtypes

In [114]:
# encoding the categorical features
categorical_feature = ['country', 'designation', 'province','region_1', 'variety', 'winery']
dict = {}
for i in categorical_feature:
  t = train_wine.groupby([i])['price'].mean().sort_values(ascending=True).index
  dict[i] = {k:i for i,k in enumerate(t.to_pandas(),0)}



In [115]:
for i in categorical_feature:
    list=[]
    for j in train_wine[i].to_pandas():
        list.append(dict[i][j])
    train_wine[i]=list

In [116]:
for i in categorical_feature:
    list=[]
    for j in test_wine[i].to_pandas():
        if j not in dict[i].keys():
            list.append(None)
        else:
            list.append(dict[i][j])
    test_wine[i]=list

In [117]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
x=SentimentIntensityAnalyzer()

In [118]:
def punctuation_clean(texL): 
    import string as st
    list=[]
    for text in texL.to_pandas():
        a=[w for w in text if w not in st.punctuation]
        list.append(''.join(a))
    return list
train_wine['description']=punctuation_clean(train_wine['description'])
test_wine['description']=punctuation_clean(test_wine['description'])
train_wine.head(2)

In [119]:
import nltk
def stopword_removal(textL):
    List=[]
    for text in textL.to_pandas():
        stopword=nltk.corpus.stopwords.words('english')
        stopword.remove('not')
        a=[w for w in nltk.word_tokenize(text) if w not in stopword]
        List.append(' '.join(a))
    return List
train_wine['description']=stopword_removal(train_wine['description'])
test_wine['description']=stopword_removal(test_wine['description'])
train_wine.head(2)

In [120]:
List=[]

for j in train_wine['description'].to_pandas():
    cp=x.polarity_scores(j)['compound']
    
    fin=1
    if cp>=0.05:
        fin=2
    elif cp>-0.05:
        fin=1
    else:
        fin=0
    
    List.append(fin)
train_wine['description']=List

In [121]:
List=[]
for j in test_wine['description'].to_pandas():
    cp=x.polarity_scores(j)['compound']
    
    fin=1
    if cp>=0.05:
        fin=2
    elif cp>-0.05:
        fin=1
    else:
        fin=0
    
    List.append(fin)
test_wine['description']=List

In [122]:
test_wine.fillna(test_wine.median(),inplace=True)

In [123]:
train_wine = train_wine.astype('float32')
test_wine=test_wine.astype('float32')

In [124]:
Y = train_wine['price']
X = train_wine.drop(['price'],axis=1)

In [125]:
from cuml.preprocessing import StandardScaler
ss = StandardScaler().fit(X)
x_scaled=ss.transform(X)

In [126]:
x_scaled

In [127]:
from cuml.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_scaled, Y, train_size=0.7, random_state=42)

In [128]:
from cuml.ensemble import RandomForestRegressor
cuml_model = RandomForestRegressor(max_features=1.0, n_bins=128,
                    min_samples_leaf=1,
                    min_samples_split=2,
                    n_estimators=100, accuracy_metric='r2')

In [129]:
cuml_model.fit(X_train,Y_train)

In [130]:
final_score = cuml_model.score(X_train,Y_train)
print(final_score)

In [131]:
pred=cuml_model.predict(X_test)
pred

In [132]:
import cuml
print(cuml.metrics.regression.mean_squared_error(Y_test,pred))
print(cuml.metrics.regression.mean_absolute_error(Y_test,pred))
print(cuml.metrics.regression.r2_score(Y_test,pred))